In [1]:
import sqlite3
conn = sqlite3.connect("nominations.db")
cursor = conn.cursor()

First let's take a look at the db's schema.

In [105]:
q0 = '''
SELECT * FROM sqlite_master;
'''

db_schema = cursor.execute(q0).fetchall()

for item in db_schema:
    print(item,'\n')
    
print(len((db_schema)))

('table', 'ceremonies', 'ceremonies', 6, 'CREATE TABLE ceremonies (\n    id INTEGER primary key,\n    Year INTEGER,\n    Host TEXT\n)') 

('table', 'nominations', 'nominations', 7, 'CREATE TABLE "nominations" (\n    id integer primary key,\n    category text,\n    nominee text,\n    movie text,\n    character text,\n    won integer,\n    ceremony_id integer,\n    foreign key (ceremony_id) REFERENCES ceremonies (id)\n)') 

('table', 'movies', 'movies', 2, 'CREATE TABLE movies(\n    id integer PRIMARY KEY,\n    movie text\n)') 

('table', 'actors', 'actors', 5, 'CREATE TABLE actors(\n    id integer PRIMARY KEY,\n    actor text\n)') 

4


Take a look at the nominations table's schema and the first ten rows.

In [82]:
q1 = '''
PRAGMA table_info(nominations);
'''

schema = cursor.execute(q1).fetchall()
schema

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'character', 'text', 0, None, 0),
 (5, 'won', 'integer', 0, None, 0),
 (6, 'ceremony_id', 'integer', 0, None, 0)]

In [75]:
q2 = '''
SELECT * FROM nominations LIMIT 10;
'''

first_ten = cursor.execute(q2).fetchall()
first_ten

for item in first_ten:
    print(item)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1)
(2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0, 1)
(3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0, 1)
(4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1, 1)
(5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0, 1)
(6, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1, 1)
(7, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0, 1)
(8, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0, 1)
(9, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0, 1)
(10, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0, 1)


In [113]:
q_drop = '''
DROP TABLE IF EXISTS ceremonies;
'''
conn.execute(q_drop)

We want to create a new table within the databse named "ceremonies". This table will have the Year and the Host only.

In [66]:
q3 = '''
    CREATE TABLE ceremonies (
    id INTEGER primary key,
    Year INTEGER,
    Host TEXT
);
'''
conn.execute(q3)

In [67]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
insert_query = '''
INSERT INTO ceremonies (Year, Host) VALUES (?, ?);
'''

# The .executemany() method allows us to perform many insert commands at once, until the list ends.
conn.executemany(insert_query, years_hosts)

In [68]:
q4 = '''
SELECT * FROM ceremonies LIMIT 10;
'''

results = cursor.execute(q4).fetchall()
results

[(1, 2010, 'Steve Martin'),
 (2, 2009, 'Hugh Jackman'),
 (3, 2008, 'Jon Stewart'),
 (4, 2007, 'Ellen DeGeneres'),
 (5, 2006, 'Jon Stewart'),
 (6, 2005, 'Chris Rock'),
 (7, 2004, 'Billy Crystal'),
 (8, 2003, 'Steve Martin'),
 (9, 2002, 'Whoopi Goldberg'),
 (10, 2001, 'Steve Martin')]

In [83]:
q5 = '''
PRAGMA table_info(ceremonies);
'''

results = cursor.execute(q5).fetchall()
results

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'Year', 'INTEGER', 0, None, 0),
 (2, 'Host', 'TEXT', 0, None, 0)]

We want to avoid editing the original table. So let's make a new nominations table, but this time we'll include the ceremony_id as a foriegn key. 

In [70]:
q6 = '''
PRAGMA foreign_keys = ON;
'''
conn.execute(q6)

In [71]:
q7 = '''
    CREATE TABLE nominations_two (
    id integer primary key,
    category text,
    nominee text,
    movie text,
    character text,
    won integer,
    ceremony_id integer,
    foreign key (ceremony_id) REFERENCES ceremonies (id)
);
'''

conn.execute(q7)

In [73]:
# Query to insert values into nominations_two
insert_query2 = '''
INSERT INTO nominations_two (category, nominee, movie, character, won, ceremony_id) VALUES (?, ?, ?, ?, ?, ?);
'''

We'll need a list of tuples for the values to be inserted using query above, we can do this by writing a query and setting the results to a list.

In [88]:
q8 = '''
SELECT nominations.category, nominations.nominee, nominations.movie, nominations.character, nominations.won, ceremonies.id
FROM nominations
INNER JOIN ceremonies
ON ceremonies.id == nominations.ceremony_id;
'''
# Returns a list of tuples
joined_nominations = cursor.execute(q8).fetchall()
joined_nominations[:3]

[('Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 ('Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1),
 ('Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  0,
  1)]

In [89]:
conn.executemany(insert_query2, joined_nominations)

 Let's check out the new table, "nominations_two" that we just created and poulated.

In [90]:
q9 = '''
SELECT * FROM nominations_two LIMIT 5;
'''
result = cursor.execute(q9).fetchall()
result[:3]

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0, 1),
 (2,
  'Actor -- Leading Role',
  'Jeff Bridges',
  'True Grit',
  'Rooster Cogburn',
  0,
  1),
 (3,
  'Actor -- Leading Role',
  'Jesse Eisenberg',
  'The Social Network',
  'Mark Zuckerberg',
  0,
  1)]

We can now change the table name of nominations_two back to nominations and replace the table.

In [91]:
q10 = '''
DROP TABLE nominations;
'''
conn.execute(q10)

q11 = '''
ALTER TABLE nominations_two RENAME TO nominations;
'''
conn.execute(q11)

We can repeat the process and create three new tables and use have their IDs cross reference each other.

In [97]:
q11 = '''
    CREATE TABLE movies(
    id integer PRIMARY KEY,
    movie text
)'''

conn.execute(q11)

insert_into_movies = "INSERT INTO movies (movie) SELECT nominations.movie FROM nominations;"

conn.execute(insert_into_movies)

In [100]:
q12 = '''
    CREATE TABLE actors(
    id integer PRIMARY KEY,
    actor text
)'''

conn.execute(q12)

insert_into_actors = "INSERT INTO actors (actor) SELECT nominations.nominee FROM nominations;"

conn.execute(insert_into_actors)

In [104]:
print(conn.execute("SELECT * FROM movies LIMIT 5").fetchall())
print(conn.execute("SELECT * FROM actors LIMIT 5").fetchall())

[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


In [111]:
q14 = "SELECT movie, nominee FROM nominations"
list_movies_actors = conn.execute(q14).fetchall()
list_movies_actors[:5]

[('Biutiful', 'Javier Bardem'),
 ('True Grit', 'Jeff Bridges'),
 ('The Social Network', 'Jesse Eisenberg'),
 ("The King's Speech", 'Colin Firth'),
 ('127 Hours', 'James Franco')]

In [114]:
# Movie & Actors relations

q13 = '''
    CREATE TABLE movies_actors(
    id integer PRIMARY KEY,
    movie_id integer REFERENCES movies (id),
    actor_id integer REFERENCES actors (id)
    );
'''

conn.execute(q13)

insert_query = '''
    INSERT INTO movies_actors (movie_id, actor_id)
    VALUES ((SELECT id FROM movies WHERE movie == ?), (SELECT id FROM actors WHERE actor == ?))
'''
conn.executemany(insert_query, list_movies_actors) # list_movies_actors MUST pass in the order of movie,actor

In [115]:
print(conn.execute("SELECT * FROM movies_actors LIMIT 5").fetchall())

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5)]
